<a href="https://colab.research.google.com/github/mansi1189/Natural-Language-Processing/blob/master/Named%20Entity%20Recognition%20using%20bert%20in%20pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from zipfile import ZipFile
import os
import pickle
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange
from seqeval.metrics import f1_score


Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
!pip3 install pytorch_pretrained_bert==0.4.0
!pip install seqeval

In [0]:

df= pd.read_csv('atis.dict.vocab.csv',header=None)
VOCAB=df.values.tolist()
VOCAB=sum(VOCAB, [])


In [0]:
dff= pd.read_csv('atis.dict.slots.csv',header=None)
slots=dff.values.tolist()
slots=sum(slots, [])

In [0]:
sen2idx = {tag: idx for idx, tag in enumerate(VOCAB)}
idx2sen = {idx: tag for idx, tag in enumerate(VOCAB)}

tag2idx = {tag: idx for idx, tag in enumerate(slots)}
idx2tag = {idx: tag for idx, tag in enumerate(slots)}


In [0]:
train_query=pd.read_csv('atis_final.csv')
train_text=train_query['tokens'].values.tolist()
train_slot=train_query['slots'].values.tolist()
train_msg=train_query['intent'].values.tolist()

In [0]:
for i in range(len(train_text)):
  train_text[i]=train_msg[i]+" "+train_text[i]
  

In [0]:
for i in range(len(train_slot)):
  train_slot[i]=train_msg[i]+" "+train_slot[i]
  

In [105]:
train_slot

['no_msg O O O O O O O B-round_trip I-round_trip O O B-fromloc.city_name O B-toloc.city_name O O B-depart_date.month_name B-depart_date.day_number I-depart_date.day_number O O O B-return_date.month_name B-return_date.day_number O',
 'no_msg O O O O O O O B-fromloc.city_name I-fromloc.city_name O O O B-toloc.city_name B-arrive_time.time_relative I-arrive_time.time_relative I-arrive_time.time_relative B-arrive_time.time I-arrive_time.time B-arrive_date.date_relative B-arrive_date.day_name O',
 'no_msg O O O O O O B-fromloc.city_name I-fromloc.city_name O B-toloc.city_name O B-depart_date.date_relative B-depart_date.day_name O O B-return_date.date_relative I-return_date.date_relative I-return_date.date_relative O',
 'no_msg O O O O O O B-meal_code I-meal_code I-meal_code O',
 'no_msg O O O O O O B-fromloc.city_name O B-toloc.city_name O O O B-meal O B-depart_date.today_relative I-depart_date.today_relative I-depart_date.today_relative I-depart_date.today_relative O',
 'no_msg O O O O B-ai

In [0]:
MAX_LEN = 75
bs = 32

In [0]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()


In [10]:
torch.cuda.get_device_name(0) 

'Tesla T4'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


100%|██████████| 231508/231508 [00:00<00:00, 1201925.19B/s]


In [107]:
tags_vals = list(tag2idx.keys())
print(tags_vals)

['B-aircraft_code', 'B-airline_code', 'B-airline_name', 'B-airport_code', 'B-airport_name', 'B-arrive_date.date_relative', 'B-arrive_date.day_name', 'B-arrive_date.day_number', 'B-arrive_date.month_name', 'B-arrive_date.today_relative', 'B-arrive_time.end_time', 'B-arrive_time.period_mod', 'B-arrive_time.period_of_day', 'B-arrive_time.start_time', 'B-arrive_time.time', 'B-arrive_time.time_relative', 'B-booking_class', 'B-city_name', 'B-class_type', 'B-compartment', 'B-connect', 'B-cost_relative', 'B-day_name', 'B-day_number', 'B-days_code', 'B-depart_date.date_relative', 'B-depart_date.day_name', 'B-depart_date.day_number', 'B-depart_date.month_name', 'B-depart_date.today_relative', 'B-depart_date.year', 'B-depart_time.end_time', 'B-depart_time.period_mod', 'B-depart_time.period_of_day', 'B-depart_time.start_time', 'B-depart_time.time', 'B-depart_time.time_relative', 'B-economy', 'B-fare_amount', 'B-fare_basis_code', 'B-flight', 'B-flight_days', 'B-flight_mod', 'B-flight_number', 'B-fl

In [108]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in train_text]
print(tokenized_texts[0:10])
len(tokenized_texts)

[['no', '_', 'ms', '##g', 'bo', '##s', 'what', 'is', 'the', 'cost', 'of', 'a', 'round', 'trip', 'flight', 'from', 'pittsburgh', 'to', 'atlanta', 'beginning', 'on', 'april', 'twenty', 'fifth', 'and', 'returning', 'on', 'may', 'sixth', 'e', '##os'], ['no', '_', 'ms', '##g', 'bo', '##s', 'now', 'i', 'need', 'a', 'flight', 'leaving', 'fort', 'worth', 'and', 'arriving', 'in', 'denver', 'no', 'later', 'than', '2', 'pm', 'next', 'monday', 'e', '##os'], ['no', '_', 'ms', '##g', 'bo', '##s', 'i', 'need', 'to', 'fly', 'from', 'kansas', 'city', 'to', 'chicago', 'leaving', 'next', 'wednesday', 'and', 'returning', 'the', 'following', 'day', 'e', '##os'], ['no', '_', 'ms', '##g', 'bo', '##s', 'what', 'is', 'the', 'meaning', 'of', 'meal', 'code', 's', 'e', '##os'], ['no', '_', 'ms', '##g', 'bo', '##s', 'show', 'me', 'all', 'flights', 'from', 'denver', 'to', 'pittsburgh', 'which', 'serve', 'a', 'meal', 'for', 'the', 'day', 'after', 'tomorrow', 'e', '##os'], ['no', '_', 'ms', '##g', 'bo', '##s', 'show'

24758

In [0]:
#pad_sequences is used to ensure that all sequences in a list have the same length. By default this is done by padding 0 in the beginning of each sequence until each sequence has the same length as the longest sequence.
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", padding="post")

In [110]:
input_ids

array([[2053, 1035, 5796, ...,    0,    0,    0],
       [2053, 1035, 5796, ...,    0,    0,    0],
       [2053, 1035, 5796, ...,    0,    0,    0],
       ...,
       [3198, 1035, 2709, ...,    0,    0,    0],
       [3198, 1035, 2709, ...,    0,    0,    0],
       [3198, 1035, 2709, ...,    0,    0,    0]])

In [0]:
tok_slot=[]

for i in train_slot:
  tok_slot.append(i.split())

In [0]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in tok_slot],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",dtype="long", truncating="post")

In [113]:
print(len(train_text))
print(len(train_slot))
print(len(tags))
print(len(input_ids))

24758
24758
24758
24758


In [0]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [115]:
print(attention_masks[:1])

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


In [0]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [117]:
len(val_inputs)

2476

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [0]:
_#TensorDataset:Dataset wrapping tensors.

#Each sample will be retrieved by indexing tensors along the first dimension.

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)


#Data loader. Combines a dataset and a sampler, and provides single- or multi-process iterators over the dataset.
# sampler (Sampler, optional) – defines the strategy to draw samples from the dataset. 
# bs=32




In [0]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [0]:
model.cuda();

In [0]:

FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)


In [0]:


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In some cases you may find that each layer of your net amplifies the gradient it receives. This causes a problem because the lower layers of the net then get huge gradients and their updates will be far too large to allow the model to learn anything.

This function ‘clips’ the norm of the gradients by scaling the gradients down by the same amount in order to reduce the norm to an acceptable level. In practice this places a limit on the size of the parameter updates.

The hope is that this will ensure that your model gets reasonably sized gradients and that the corresponding updates will allow the model to learn.

In [124]:
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    print("tr loss ",tr_loss)
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    print("eval_example ",nb_eval_examples)
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.09776385831733474
tr loss  68.14140924718231
eval_example  2476
Validation loss: 0.023813401092178166
Validation Accuracy: 0.9949697293447295


Epoch:  20%|██        | 1/5 [06:54<27:38, 414.62s/it]

F1-Score: 0.9006028525216879
Train loss: 0.018859301488799773
tr loss  13.144933137693442
eval_example  2476
Validation loss: 0.012351521932615492
Validation Accuracy: 0.9970548433048441


Epoch:  40%|████      | 2/5 [13:51<20:45, 415.17s/it]

...

F1-Score: 0.9550258765216123
Train loss: 0.007385572163922441
tr loss  5.147743798253941
eval_example  2476
Validation loss: 0.006789154373855593
Validation Accuracy: 0.998358262108263


Epoch:  80%|████████  | 4/5 [27:43<06:55, 415.63s/it]

F1-Score: 0.9686884169038756
Train loss: 0.005184359029729629
tr loss  3.6134982437215513
eval_example  2476
Validation loss: 0.0060634969416311765
Validation Accuracy: 0.9985274216524224


Epoch: 100%|██████████| 5/5 [34:38<00:00, 415.52s/it]

F1-Score: 0.9724757246112288


In [0]:
def get_output(msg:str,text: str):
  text=msg+" BOS "+text+" EOS"

  tokenized_texts = [tokenizer.tokenize(text)]
  input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", padding="post")
  attention_masks = [[float(i>0) for i in ii] for ii in input_ids]  

  test_inputs=torch.tensor(input_ids)

  test_mask=torch.tensor(attention_masks)

  test_data = TensorDataset(test_inputs, test_mask)
  test_sampler = RandomSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)

  predictions=[]



  model.eval()
  for batch in test_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask = batch
  #     print(b_input_ids)
      with torch.no_grad():

          logits = model(b_input_ids, token_type_ids=None,
                         attention_mask=b_input_mask)

      logits = logits.detach().cpu().numpy()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      label_ids = b_labels.to('cpu').numpy()
  pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]  



  labels = []
  for valid,label in zip(tokenized_texts,pred_tags):
      if valid:
          labels.append(label)

  text=text.split()       
  output=[]
  for i in range(len(text)):
    output.append("word: {} Tag: {}".format(text[i],pred_tags[0][i]))
  # output = {word:{"tag":label} for word,label in zip(tokenized_texts,labels)}

  return output


  

  
  
  

# no_msg
# ask_src
# ask_des
# ask_dep_date
# ask_return_date

In [153]:
get_output("ask_return_date","on june 5th")

['word: ask_return_date Tag: ask_return_date',
 'word: BOS Tag: O',
 'word: on Tag: O',
 'word: june Tag: B-return_date.month_name',
 'word: 5th Tag: B-return_date.day_number',
 'word: EOS Tag: O']